In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("../Data/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
X = df.drop(['bought_insurance'], axis=1)
y = df.bought_insurance

In [161]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [162]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

# <span style='color:MediumVioletRed'>Creating the myNN class</span>

In [27]:
def sigmoid_numpy(X):
    return 1 / (1 + np.exp(-X))

In [28]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i, epsilon) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [99]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_threshold)

    def predict(self, X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        probability_values = sigmoid_numpy(weighted_sum)
        return probability_values

    def score(self, X_test, y_test):
        probability_values = self.predict(X_test)
        binary_predictions = (probability_values >= 0.5).astype(int)
        true_predictions = np.sum(binary_predictions == y_test)
        return true_predictions/y_test.size

    def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
        # w1, w2, bias
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)

        for i in range(epochs):
            weighted_sum = w1*age + w2*affordibility + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            #calculating the loss
            loss = log_loss(y_true, y_predicted)
            #calculating derivatives
            w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_true))
            w2d = (1/n) * np.dot(np.transpose(affordibility), (y_predicted - y_true))
            bias_d = np.mean(y_predicted - y_true)
            #adjusting weights
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
        
            #printing updates
            if i%50==0:
                print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")
        
            if loss<=loss_threshold:
                print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")
                break

        return w1, w2, bias

# <span style='color:MediumVioletRed'>Comparing Tensorflow and Custom Neural Network</span>

## <span  style="color:red">Tensor Flow</span>

In [24]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics= ['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Epoch 1/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - accuracy: 0.5000 - loss: 0.7272
Epoch 2/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7267
Epoch 3/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5000 - loss: 0.7263
Epoch 4/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5000 - loss: 0.7259
Epoch 5/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7254
Epoch 6/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5000 - loss: 0.7250
Epoch 7/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7246
Epoch 8/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7242
Epoch 9/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7237
Epoch 10/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.5000 - loss: 0.7233
Epoch 11/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5000 - loss: 0.7229
Epoch 12/5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy

In [146]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.1204925],
        [1.7509859]], dtype=float32),
 array([-3.3396778], dtype=float32))

In [150]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 1.0000 - loss: 0.3216


[0.32157817482948303, 1.0]

In [147]:
y_predicted_tf = model.predict(X_test_scaled)
y_predicted_tf

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


array([[0.3720559 ],
       [0.38646796],
       [0.7822427 ],
       [0.7151133 ],
       [0.11832901],
       [0.7642953 ]], dtype=float32)

In [148]:
y_test

6     0
0     0
5     1
14    1
21    0
25    1
Name: bought_insurance, dtype: int64

## <span  style="color:red">Custom Model</span>

In [163]:
custom_model = myNN()

In [178]:
custom_model.fit(X_train_scaled, y_train, epochs = 5000, loss_threshold = 0.3901)

Epoch:0, w1:0.9662203069036812, w2:0.9301601323389443, bias:-0.13965993618686595, loss:0.758009630012808
Epoch:50, w1:1.4491224685096329, w2:1.1682851656285322, bias:-1.4733368647627307, loss:0.5504113177048761
Epoch:100, w1:2.124965979496889, w2:1.4535581659190615, bias:-1.9878564009092836, loss:0.5177802567557316
Epoch:150, w1:2.7558096322403656, w2:1.5908808682785782, bias:-2.363477273756611, loss:0.49529237936588744
Epoch:200, w1:3.3356898067887606, w2:1.6612357624214567, bias:-2.6620902746170185, loss:0.47798885104980665
Epoch:250, w1:3.865240925601195, w2:1.6995402064865597, bias:-2.9127646883109097, loss:0.46414108617135935
Epoch:300, w1:4.347682764743223, w2:1.7222409981415603, bias:-3.130721661778457, loss:0.4528668039037049
Epoch:350, w1:4.787139208753815, w2:1.7374425399594653, bias:-3.324602548644628, loss:0.44359537129444226
Epoch:400, w1:5.187874663479474, w2:1.749199231025196, bias:-3.4996621533671295, loss:0.43591375381691916
Epoch:450, w1:5.553946611327524, w2:1.759520

In [179]:
custom_model.w1, custom_model.w2, custom_model.bias

(np.float64(10.553515079586376),
 np.float64(2.0576449765145424),
 np.float64(-5.995496588943619))

In [180]:
custom_model.score(X_test_scaled, y_test)

np.float64(1.0)

In [181]:
y_predicted = custom_model.predict(X_test_scaled)
y_predicted

23    0.692372
0     0.165746
7     0.583367
21    0.037270
14    0.774408
4     0.714383
dtype: float64

In [168]:
y_test

23    1
0     0
7     1
21    0
14    1
4     1
Name: bought_insurance, dtype: int64